In [1]:
import sys
sys.path.append("../")

import numpy as np
import time
import os
import ctypes

In [2]:
_path = os.getcwd()
lib = np.ctypeslib.load_library('../lib/trial.so', _path)
cxtbx = lib.xTBx
cxtbx.restype  = ctypes.c_int
cxtbx.argtypes = [ctypes.c_int,
                  ctypes.c_int,
                  ctypes.c_double,
                  np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                  np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                  np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED')]

In [3]:
x = np.random.rand(24).reshape(4, 6)
B = np.random.rand(16).reshape(4, 4)
np.dot(x.T, np.dot(B, x))

array([[ 1.60851195,  0.97178857,  1.9500249 ,  2.10651045,  2.51152474,
         1.7679169 ],
       [ 0.77549115,  0.47865407,  0.94233361,  1.02929737,  1.19366188,
         0.86624038],
       [ 1.61852263,  0.93521952,  1.84709586,  2.07247391,  2.29014617,
         1.76682764],
       [ 1.84616528,  1.0246578 ,  2.05302412,  2.31945414,  2.54479811,
         1.98522826],
       [ 2.0234954 ,  1.18413078,  2.31295624,  2.60309091,  2.84739029,
         2.22105509],
       [ 1.60176492,  0.89200903,  1.79524963,  2.01621254,  2.24077576,
         1.7217672 ]])

In [4]:
nrowA  = x.shape[0]
ncolA  = x.shape[1]
nrowB  = B.shape[0]
ncolB  = B.shape[1]
C = np.zeros(ncolA * ncolA)
print(C)
success = cxtbx(nrowA, ncolA, 1.0, x.reshape(-1,), B.reshape(-1,), C)
C.reshape(ncolA, ncolA)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


array([[ 1.60851195,  0.97178857,  1.9500249 ,  2.10651045,  2.51152474,
         1.7679169 ],
       [ 0.77549115,  0.47865407,  0.94233361,  1.02929737,  1.19366188,
         0.86624038],
       [ 1.61852263,  0.93521952,  1.84709586,  2.07247391,  2.29014617,
         1.76682764],
       [ 1.84616528,  1.0246578 ,  2.05302412,  2.31945414,  2.54479811,
         1.98522826],
       [ 2.0234954 ,  1.18413078,  2.31295624,  2.60309091,  2.84739029,
         2.22105509],
       [ 1.60176492,  0.89200903,  1.79524963,  2.01621254,  2.24077576,
         1.7217672 ]])

In [5]:
cinv = lib.matloginv
cinv.restype = ctypes.c_double
cinv.argtypes = [ctypes.c_int,
                 np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED')]

In [6]:
matrixSize = 10 
D = np.random.rand(matrixSize,matrixSize)
Dcov = np.dot(D, D.transpose())
Dcov

array([[ 2.8805317 ,  1.55539876,  2.82170139,  1.18823801,  2.12356452,
         2.62351024,  1.11599178,  2.60105646,  2.11883332,  2.24191602],
       [ 1.55539876,  1.50575531,  1.40397564,  0.98168949,  1.58367671,
         2.08215347,  1.13177415,  1.93089844,  1.86349851,  1.56879875],
       [ 2.82170139,  1.40397564,  3.87951871,  1.44583941,  2.68888515,
         3.19050929,  1.77493283,  2.66232072,  2.41472686,  3.30226789],
       [ 1.18823801,  0.98168949,  1.44583941,  1.02218375,  1.39992392,
         1.74044662,  1.12900675,  1.47800745,  1.44322235,  1.82792187],
       [ 2.12356452,  1.58367671,  2.68888515,  1.39992392,  3.36543685,
         3.41469017,  1.35139525,  2.71886334,  2.20445072,  2.86344595],
       [ 2.62351024,  2.08215347,  3.19050929,  1.74044662,  3.41469017,
         3.97263037,  2.00333352,  3.24636668,  3.00452051,  3.40132552],
       [ 1.11599178,  1.13177415,  1.77493283,  1.12900675,  1.35139525,
         2.00333352,  1.75274486,  1.63042742

In [7]:
res = Dcov.T.reshape(-1, )
logdet = cinv(matrixSize, res)
res.reshape(matrixSize, matrixSize).T[5, 2]

27.310322413640282

In [8]:
logdet

-11.318059678151315

In [9]:
np.linalg.inv(Dcov)[5, 2]

27.310322413639863

In [10]:
np.linalg.slogdet(Dcov)

(1.0, -11.318059678151315)